In [1]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 15 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=7efcec573817591ee598252cd4da72b6d3da745a3dfdea1e3e9b9fb242d721e9
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpkz9ole_0
  JVM stdout: /tmp/tmpkz9ole_0/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpkz9ole_0/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_zun6de
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [ ]:
test, train = dataSetup('australian')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
train.shape

(517, 15)

In [ ]:
x = train.columns
y = "class"
x.remove(y)

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220501_194151_model_12


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,38.0,38.0,27392.0,8.0,10.0,9.894737,36.0,60.0,52.5




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.014995858433689582
RMSE: 0.12245757809825238
LogLoss: 0.08839945577475179
Mean Per-Class Error: 0.004149377593360996
AUC: 0.9998646942089122
AUCPR: 0.9998482802817796
Gini: 0.9997293884178244

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5000154785811852: 


,,0,1,Error,Rate
0,0,276.0,0.0,0.0,(0.0/276.0)
1,1,2.0,239.0,0.0083,(2.0/241.0)
2,Total,278.0,239.0,0.0039,(2.0/517.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.891654,0.016162,0.894231,0.903846,0.864078,0.902913,0.893204
1,auc,0.930138,0.015729,0.932963,0.942286,0.903065,0.939922,0.932453
2,err,0.108346,0.016162,0.105769,0.096154,0.135922,0.097087,0.106796
3,err_count,11.200000,1.643168,11.000000,10.000000,14.000000,10.000000,11.000000
4,f0point5,0.882858,0.035738,0.870370,0.923695,0.829875,0.904523,0.885827
5,f1,0.883480,0.020112,0.895238,0.901961,0.851064,0.878049,0.891089
6,f2,0.885130,0.025671,0.921569,0.881226,0.873362,0.853081,0.896414
7,lift_top_group,2.157300,0.178511,2.080000,1.962264,2.288889,2.395349,2.060000
8,logloss,0.335945,0.049360,0.322877,0.311590,0.421369,0.295628,0.328263
9,max_per_class_error,0.142279,0.019816,0.148148,0.132075,0.155172,0.162791,0.113208



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-05-01 19:43:05,15.705 sec,0.0,0.498853,0.690854,0.500000,0.466151,1.000000,0.533849
1,,2022-05-01 19:43:05,15.728 sec,5.0,0.364989,0.449619,0.984725,0.983206,2.145228,0.063830
2,,2022-05-01 19:43:05,15.749 sec,10.0,0.286998,0.323025,0.989258,0.988708,2.145228,0.050290
3,,2022-05-01 19:43:05,15.772 sec,15.0,0.235141,0.242363,0.994234,0.993824,2.145228,0.038685
4,,2022-05-01 19:43:05,15.806 sec,20.0,0.203880,0.191170,0.995572,0.995298,2.145228,0.034816
5,,2022-05-01 19:43:05,15.827 sec,25.0,0.177290,0.152225,0.997429,0.997196,2.145228,0.025145
6,,2022-05-01 19:43:05,15.850 sec,30.0,0.156862,0.124352,0.998512,0.998377,2.145228,0.017408
7,,2022-05-01 19:43:05,15.871 sec,35.0,0.134398,0.099900,0.999714,0.999683,2.145228,0.005803
8,,2022-05-01 19:43:05,15.887 sec,38.0,0.122458,0.088399,0.999865,0.999848,2.145228,0.003868



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,A8,237.219482,1.000000,0.409895
1,A2,64.962967,0.273852,0.112250
2,A7,46.868595,0.197575,0.080985
3,A10,44.078163,0.185812,0.076163
4,A9,39.134239,0.164971,0.067621
5,A3,36.218597,0.152680,0.062583
6,A13,31.877439,0.134380,0.055081
7,A5,23.825485,0.100436,0.041168
8,A14,22.998901,0.096952,0.039740
9,A6,11.059758,0.046622,0.019110


In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_grid_1_AutoML_1_20220501_194151_model_12,0.932257,0.323185,0.932824,0.115273,0.306901,0.094188
GBM_2_AutoML_1_20220501_194151,0.930543,0.323583,0.918193,0.119919,0.306115,0.0937062
GBM_grid_1_AutoML_1_20220501_194151_model_30,0.93016,0.327165,0.923682,0.116588,0.308705,0.0952991
StackedEnsemble_BestOfFamily_5_AutoML_1_20220501_194151,0.930122,0.323464,0.919679,0.117844,0.305474,0.0933143
GBM_grid_1_AutoML_1_20220501_194151_model_53,0.929949,0.331998,0.92295,0.123865,0.314607,0.0989775
StackedEnsemble_BestOfFamily_7_AutoML_1_20220501_194151,0.929731,0.326239,0.910078,0.116558,0.306509,0.093948
StackedEnsemble_AllModels_2_AutoML_1_20220501_194151,0.927341,0.332422,0.900587,0.122256,0.309893,0.0960335
GBM_grid_1_AutoML_1_20220501_194151_model_51,0.927168,0.335675,0.925178,0.121234,0.315523,0.0995547
GBM_grid_1_AutoML_1_20220501_194151_model_42,0.927153,0.330671,0.925912,0.120475,0.310462,0.0963867
StackedEnsemble_BestOfFamily_2_AutoML_1_20220501_194151,0.92704,0.333224,0.905041,0.118107,0.307038,0.0942723
